## Science case, Hinkley et al.

In [1]:
from heeps.config import read_config, update_config
from heeps.contrast import background
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import fit_zer, remove_zernike
import numpy as np
import os  
from astropy.io import fits
from copy import deepcopy
import proper

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
filename = 'cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s_ncpa_%s_%s.fits'
tag = {'L': 'LM', 'N2': 'N'}
rep = {'L': 'rep_6_-0h30', 'N2': 'rep_5_-0h30'}
npupils = {'L': 285, 'N2': 119}
sigLFs = {'L': 13e-9, 'N2': 13e-9*2}
sigHFs = {'L': 3e-9, 'N2': 3e-9*2}
nzer = 100
G = 0.4
dit = 0.1     # DIT = 0.1s !!!
freq = 10#3   # correction every 10 frames (1Hz)
lag = 10#3    # lag every 10 frames (1s)

In [3]:
# avoid redundancy
scao_cases = np.arange(1,25)
print('scao_cases   =', scao_cases)
talbot_cases = np.tile(np.arange(1,11), 3)[:24]
print('talbot_cases =', talbot_cases)
cbw_cases = np.concatenate([np.arange(1,5).repeat(3), np.arange(5,11).repeat(2)])
print('cbw_cases    =', cbw_cases)
wv_cases = np.tile(np.arange(1,7), 4)
print('wv_cases     =', np.append(wv_cases, 10)) # last '10' digit in printing for alignment/comparison purpose only

scao_cases   = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
talbot_cases = [ 1  2  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9 10  1  2  3  4]
cbw_cases    = [ 1  1  1  2  2  2  3  3  3  4  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
wv_cases     = [ 1  2  3  4  5  6  1  2  3  4  5  6  1  2  3  4  5  6  1  2  3  4  5  6
 10]


In [4]:
# create 24x 10 min SCAO cubes
filename = 'dfull/cube_Dfull_20240313_default_D_%s_600s_100ms_%s_%s_%s.fits'
scao_names = [
    'smedian_z30d_Kmag7.0_v1',
    'smedian_z30d_Kmag7.0_v2',
    'smedian_z30d_Kmag7.5_v1',
    'smedian_z30d_Kmag7.5_v2',
    'smedian_z30d_Kmag8.0_v1',
    'smedian_z30d_Kmag8.0_v2',
    'smedian_z30d_Kmag8.5_v1',
    'smedian_z30d_Kmag8.5_v2',
    'sQ2_z30d_Kmag7.0_v1',
    'sQ2_z30d_Kmag7.0_v2',
    'sQ2_z30d_Kmag7.5_v1',
    'sQ2_z30d_Kmag7.5_v2',
    'sQ2_z30d_Kmag8.0_v1',
    'sQ2_z30d_Kmag8.0_v2',
    'sQ2_z30d_Kmag8.5_v1',
    'sQ2_z30d_Kmag8.5_v2',
    'sQ3_z30d_Kmag7.0_v1',
    'sQ3_z30d_Kmag7.0_v2',
    'sQ3_z30d_Kmag7.5_v1',
    'sQ3_z30d_Kmag7.5_v2',
    'sQ3_z30d_Kmag8.0_v1',
    'sQ3_z30d_Kmag8.0_v2',
    'sQ3_z30d_Kmag8.5_v1',
    'sQ3_z30d_Kmag8.5_v2',    
]

In [5]:
# create 24x 10 min Talbot cubes
band = 'L'
for i in range(24):
    write_name = 'cbw/20221006/talbot/%s_10min_v%s.fits'%(tag[band], i+1)
    if os.path.isfile(write_name):
        print(write_name + ' already exists.')
    else:
        talbot = fits.getdata('cbw/20221006/talbot/%s_rep_%s_-0h30.fits'%(tag[band], talbot_cases[i]))[:2000].repeat(3,0)
        print('writing: ' + write_name)
        fits.writeto(write_name, np.float32(talbot))

cbw/20221006/talbot/LM_10min_v1.fits already exists.
cbw/20221006/talbot/LM_10min_v2.fits already exists.
cbw/20221006/talbot/LM_10min_v3.fits already exists.
cbw/20221006/talbot/LM_10min_v4.fits already exists.
cbw/20221006/talbot/LM_10min_v5.fits already exists.
cbw/20221006/talbot/LM_10min_v6.fits already exists.
cbw/20221006/talbot/LM_10min_v7.fits already exists.
cbw/20221006/talbot/LM_10min_v8.fits already exists.
cbw/20221006/talbot/LM_10min_v9.fits already exists.
cbw/20221006/talbot/LM_10min_v10.fits already exists.
cbw/20221006/talbot/LM_10min_v11.fits already exists.
cbw/20221006/talbot/LM_10min_v12.fits already exists.
cbw/20221006/talbot/LM_10min_v13.fits already exists.
cbw/20221006/talbot/LM_10min_v14.fits already exists.
cbw/20221006/talbot/LM_10min_v15.fits already exists.
cbw/20221006/talbot/LM_10min_v16.fits already exists.
cbw/20221006/talbot/LM_10min_v17.fits already exists.
cbw/20221006/talbot/LM_10min_v18.fits already exists.
cbw/20221006/talbot/LM_10min_v19.fits

In [6]:
# create 6x 10 min WV cubes (will be repeated 4 times each)
band = 'L'
wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_%s_%s_WVonly.fits'%(band, npupils[band]))
wv_10min_name = 'wv/cube_Cbasic_20210601_600s_100ms_%s_%s_WVonly'%(band, npupils[band]) + '_v%s.fits'
for i in range(6):
    write_name = wv_10min_name%(i+1)
    if os.path.isfile(write_name):
        print(write_name + ' already exists.')
    else:
        wv_10min = wv[i*nframes:(i+1)*nframes].repeat(samp,0)
        print('writing: ' + write_name)
        fits.writeto(write_name, np.float32(wv_10min))

wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v1.fits already exists.
wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v2.fits already exists.
wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v3.fits already exists.
wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v4.fits already exists.
wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v5.fits already exists.
wv/cube_Cbasic_20210601_600s_100ms_L_285_WVonly_v6.fits already exists.


In [7]:
def load_ncpa_10min(i, band):
    print(scao_cases[i], cbw_cases[i], wv_cases[i])
    npupil = npupils[band]
    scao = fits.getdata(filename%(scao_names[i], 'scao_only', band, npupil))
    cbw = fits.getdata('cbw/20221006/ncpa/%s_rep_%s_-0h30.fits'%(tag[band], cbw_cases[i]))[:2000].repeat(3,0)
    wv = fits.getdata('wv/cube_Cbasic_20210601_600s_100ms_%s_%s_WVonly'%(band, npupil) + '_v%s.fits'%wv_cases[i])
    pup = fits.getdata('cfull/mask_Cfull_20220512_%s_%s.fits'%(band, npupil))
    pup[pup < .5] = 0
    return pup, scao + cbw + wv

def load_zpols(pup, ncpa, ncpa_name, npupil):
    zpols_name = ncpa_name[:-5] + '_zpols_%s.fits'%nzer
    try:
        zpols = fits.getdata(zpols_name)
        print('    getdata ' + zpols_name)
    except FileNotFoundError:
        print('    writeto ' + zpols_name)
        zpols = multiCPU(fit_zer, posargs=[pup, npupil/2, nzer], 
            posvars=[ncpa], case='get zpols')
        fits.writeto(zpols_name, np.float32(zpols))
    return zpols

def load_zpols_integ(pup, ncpa, ncpa_name, npupil, sigLF, sigHF, freq, lag):
    zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_freq_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freq, G, lag, sigLF, sigHF)
    if os.path.isfile(zpols_integ_name):
        zpols_integ = fits.getdata(zpols_integ_name)
        print('  getdata ' + zpols_integ_name)
    else:
        print('  write to ' + zpols_integ_name)
        zpols = load_zpols(pup, ncpa, ncpa_name, npupil)
        zpols_integ = np.zeros(zpols.shape)
        nframes = len(zpols)
        # piston
        zpols_integ[:,0] = zpols[:,0]
        # tip-tilt, and higher modes
        for m, sig in zip([range(1,3), range(3,nzer)], [sigLF, sigHF]):
            for n in range(freq+lag, nframes, freq):
                error = np.mean(zpols[n-freq-lag:n-lag,m] - zpols_integ[n-1,m], 0) + np.random.normal(0, sig, (1, len(m))) 
                zpols_integ[n:n+freq,m] = zpols_integ[n-1,m] + G*error
        fits.writeto(zpols_integ_name, np.float32(zpols_integ))
    return zpols_integ

def save_ncpa_10min(i, band, freq, lag):
    npupil = npupils[band]
    ncpa_name = filename%(scao_names[i], 'all_ncpa', band, npupil)
    if os.path.isfile(ncpa_name):
        print('file already exists: ' + ncpa_name)
    else:
        print('write to ' + ncpa_name)
        pup, ncpa = load_ncpa_10min(i, band)
        sigLF = sigLFs[band]
        sigHF = sigHFs[band]
        zpols_integ = load_zpols_integ(pup, ncpa, ncpa_name, npupil, sigLF, sigHF, freq, lag)
        wf = proper.prop_begin(1, 1, npupil, 1) # initial wavefront
        _, HSF = multiCPU(remove_zernike, multi_out=True, verbose=True,
            posargs=[deepcopy(wf), pup],
            posvars=[ncpa, zpols_integ])
        fits.writeto(ncpa_name, np.float32(HSF))

In [8]:
# start ncpa correction
for i in np.arange(24):
    for band in ['L']:#, 'N2']:
        save_ncpa_10min(i, band, freq, lag)

file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag7.0_v1_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag7.0_v2_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag7.5_v1_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag7.5_v2_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag8.0_v1_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag8.0_v2_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag8.5_v1_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_smedian_z30d_Kmag8.5_v2_600s_100ms_all_ncpa_L_285.fits
file already exists: dfull/cube_Dfull_20240313_default_D_sQ2_z30d_Kmag7.0_v1_600s_100ms_all_ncpa_L_285.f

## add star mag and background

In [9]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/output_files/exeter_all_effects')))
mags = [7, 7, 7.5, 7.5, 8, 8, 8.5, 8.5, 7, 7, 7.5, 7.5, 8, 8, 8.5, 8.5, 7, 7, 7.5, 7.5, 8, 8, 8.5, 8.5]
for scao_name, mag in zip(scao_names, mags):
    loadname = scao_name + '/%s_PSF_L_CVC.fits'
    savename = '%s_PSF_L_CVC_with_bkg_' + '%s.fits'%scao_name
    psf_ON = fits.getdata(loadname%'onaxis')
    psf_OFF = fits.getdata(loadname%'offaxis')
    header_ON = fits.getheader(loadname%'onaxis')
    conf = update_config(**dict(read_config(), band='L', mode='CVC', dit=dit, mag=mag))
    print(savename%'onaxis')
    on, off = background(psf_ON, psf_OFF, header=header_ON, seed=np.random.randint(1e6), verbose=True, **conf)
    fits.writeto(savename%'onaxis', np.float32(on), overwrite=True)
    fits.writeto(savename%'offaxis', np.float32(off), overwrite=True)

onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag7.0_v1.fits
   dit=100ms, thruput=0.7738, mask_trans=0.8532,
   mag=7, star_signal=1.43e+10, bckg_noise=5.86e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag7.0_v2.fits
   dit=100ms, thruput=0.7700, mask_trans=0.8532,
   mag=7, star_signal=1.43e+10, bckg_noise=5.83e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag7.5_v1.fits
   dit=100ms, thruput=0.7675, mask_trans=0.8532,
   mag=7.5, star_signal=9.00e+09, bckg_noise=5.81e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag7.5_v2.fits
   dit=100ms, thruput=0.7783, mask_trans=0.8532,
   mag=7.5, star_signal=9.00e+09, bckg_noise=5.90e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag8.0_v1.fits
   dit=100ms, thruput=0.7840, mask_trans=0.8532,
   mag=8, star_signal=5.68e+09, bckg_noise=5.94e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag8.0_v2.fits
   dit=100ms, thruput=0.7777, mask_trans=0.8532,
   mag=8, star_signal=5.68e+09, bckg_noise=5.89e+06
onaxis_PSF_L_CVC_with_bkg_smedian_z30d_Kmag8.5_v1.fits
   di